# Win Probability based on the objectives taken by professional League of Legends teams
This report will outline my mini project for the class "Machine Learning for Media-Experiences" at Aalborg University Copenhagen for the Medialogy Masters class. In this mini project I will be looking at the professional scene of the MOBA game "League of Legends", and more specifically the different neutral objectives and how much they affect the killing team chances of winning.

## Background and details about the objectives in the game
In the game of "League of Legends" there are many factors that influence which team wins, each of which are accumulated differently and mostly do not 

# Things to look at
Class applications something to remove bias
Principal component a... (TSNE)
optimizers
every factor relating to it

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

Matplotlib is building the font cache; this may take a moment.


In [87]:
df = pd.read_csv("data/2023_LoL_esports_match_data_from_OraclesElixir.csv")

# Usually you would want to remove the data point if they are empty, however in this case I wanna do the opposite
# I remove all the values that have a player, as I am only interested in the teams performance, not the individual.
df = df[df['playername'].isna()]

# As there are a lot of data in this csv file, I am gonna trim most of them off here.
# Most are related to player specific stats, which we are not that interested in.
# Also some data points that are empty that get removed here
df.drop(columns=['playername', 'playerid', 'firstbloodkill','firstbloodassist','firstbloodvictim','dragons (type unknown)','total cs','monsterkillsownjungle','monsterkillsenemyjungle','teamkills','teamdeaths','damageshare'], inplace=True)

df.drop(columns=['wardsplaced','wpm','wardskilled','wcpm','controlwardsbought','visionscore','vspm'], inplace=True)

df.drop(columns=['gameid','datacompleteness','url','year','split','playoffs','date','game','patch','participantid','position','teamid','champion'],inplace=True)

df.replace("", float("NaN"), inplace=True) 
df.dropna(how='all',axis=1, inplace=True)

df.head()

C:\Users\Ejer\AppData\Local\Temp\ipykernel_20240\3637227049.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/2023_LoL_esports_match_data_from_OraclesElixir.csv")


,league,side,teamname,ban1,ban2,ban3,ban4,ban5,gamelength,result,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
10,LFL2,Blue,Klanik Esport,Sylas,Caitlyn,Wukong,Akali,Yone,2612,1,...,535.0,-530.0,-1671.0,-37.0,0.0,0.0,1.0,1.0,1.0,0.0
11,LFL2,Red,MS Company,Galio,Lucian,Fiora,Viktor,Azir,2612,0,...,498.0,530.0,1671.0,37.0,1.0,1.0,0.0,0.0,0.0,1.0
22,LFL2,Blue,beGenius ESC,Fiora,Kindred,Sejuani,Nautilus,Leona,2436,0,...,532.0,673.0,530.0,-34.0,4.0,6.0,3.0,3.0,4.0,4.0
23,LFL2,Red,ViV Esport,Heimerdinger,Wukong,Akali,Syndra,Sylas,2436,1,...,498.0,-673.0,-530.0,34.0,3.0,4.0,4.0,4.0,6.0,3.0
34,LFL2,Blue,Team du Sud,Zac,Sylas,Sejuani,Jarvan IV,Ornn,1980,1,...,452.0,-1901.0,-763.0,58.0,2.0,1.0,4.0,4.0,6.0,2.0


Removing many of the columns that hold unimportant stats for the outcome of the game in general, as well as stats portraying information that is less relevant to the teams performance, there is still a lot of columns. I now choose to focus on the columns that relate to objectives, but I keep the other dataset saved, this way I can always go back and get it if it becomes relevant for something further down the line.

The columns I will be choosing to keep are:
 - Dragons killed per team
 - Rift heralds killed by team
 - Barons killed by team
 - Turrets/inhibitors killed by team

In [100]:
df_objectives = df.copy()

df_objectives.drop(columns=['league','teamname','ban1','ban2','ban3','ban4','ban5'], inplace=True)
df_objectives.drop(columns=['kills','deaths','assists','doublekills','triplekills','quadrakills','pentakills','firstblood','team kpm','ckpm','opp_dragons','opp_elementaldrakes','opp_heralds','opp_elders','opp_barons','opp_towers','opp_turretplates','opp_inhibitors'],inplace=True)
df_objectives.drop(columns=['dragons','infernals','clouds','mountains','hextechs','oceans','chemtechs'],inplace=True)
df_objectives.drop(columns=['damagetochampions','dpm','damagetakenperminute','damagemitigatedperminute','totalgold','earnedgold','earned gpm','earnedgoldshare','goldspent','gspd','minionkills','monsterkills','cspm','goldat10','xpat10','csat10','opp_goldat10','opp_xpat10','opp_csat10','csdiffat10','killsat10','assistsat10','deathsat10','opp_killsat10','opp_assistsat10','opp_deathsat10','goldat15','xpat15','csat15','opp_goldat15','opp_xpat15','opp_csat15','csdiffat15','killsat15','assistsat15','deathsat15','opp_killsat15','opp_assistsat15','opp_deathsat15'], inplace=True)

df_objectives.head()

,side,gamelength,result,firstdragon,elementaldrakes,elders,firstherald,heralds,firstbaron,barons,firsttower,towers,firstmidtower,firsttothreetowers,turretplates,inhibitors,golddiffat10,xpdiffat10,golddiffat15,xpdiffat15
10,Blue,2612,1,0.0,4.0,0.0,1.0,2.0,1.0,1.0,1.0,11.0,1.0,1.0,4.0,2.0,75.0,-156.0,-530.0,-1671.0
11,Red,2612,0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-75.0,156.0,530.0,1671.0
22,Blue,2436,0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,0.0,5.0,1.0,1.0,6.0,0.0,-361.0,282.0,673.0,530.0
23,Red,2436,1,1.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,11.0,0.0,0.0,2.0,3.0,361.0,-282.0,-673.0,-530.0
34,Blue,1980,1,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,3.0,1.0,-1001.0,-1748.0,-1901.0,-763.0


In [96]:
df_objectives.dropna(inplace=True)


In [101]:
df_objectives_blue = df_objectives[df_objectives['side'] == 'Blue']

df_objectives_blue.head()

,side,gamelength,result,firstdragon,elementaldrakes,elders,firstherald,heralds,firstbaron,barons,firsttower,towers,firstmidtower,firsttothreetowers,turretplates,inhibitors,golddiffat10,xpdiffat10,golddiffat15,xpdiffat15
10,Blue,2612,1,0.0,4.0,0.0,1.0,2.0,1.0,1.0,1.0,11.0,1.0,1.0,4.0,2.0,75.0,-156.0,-530.0,-1671.0
22,Blue,2436,0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,0.0,5.0,1.0,1.0,6.0,0.0,-361.0,282.0,673.0,530.0
34,Blue,1980,1,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,3.0,1.0,-1001.0,-1748.0,-1901.0,-763.0
46,Blue,1887,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,-497.0,237.0,504.0,780.0
58,Blue,2100,1,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,10.0,1.0,1.0,4.0,2.0,525.0,-920.0,-786.0,-2171.0


In [102]:
df_objectives_red = df_objectives[df_objectives['side'] == 'Red']

df_objectives_red.head()

,side,gamelength,result,firstdragon,elementaldrakes,elders,firstherald,heralds,firstbaron,barons,firsttower,towers,firstmidtower,firsttothreetowers,turretplates,inhibitors,golddiffat10,xpdiffat10,golddiffat15,xpdiffat15
11,Red,2612,0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-75.0,156.0,530.0,1671.0
23,Red,2436,1,1.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,11.0,0.0,0.0,2.0,3.0,361.0,-282.0,-673.0,-530.0
35,Red,1980,0,1.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,4.0,0.0,1.0,6.0,0.0,1001.0,1748.0,1901.0,763.0
47,Red,1887,1,1.0,4.0,0.0,0.0,1.0,1.0,1.0,0.0,9.0,1.0,1.0,1.0,1.0,497.0,-237.0,-504.0,-780.0
59,Red,2100,0,1.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,-525.0,920.0,786.0,2171.0


This cuts the data set all the way down to 25 columns! Which is a 1/5th of the original dataset, including it already being 1/6th of the rows as well. Lastly I get some information about the dataset, to see which kind of data type each of the rows have.

In [77]:
# Assuming you have a DataFrame called 'df' with a column 'side' containing "blue" and "red" values
df_objectives['side'] = df_objectives['side'].map({'Blue': 0, 'Red': 1})


In [83]:
# Get information about the dataframe
#df.drop(df.columns[[0]], axis=1, inplace=True)
df_objectives.info()


<class 'pandas.core.frame.DataFrame'>
Index: 18448 entries, 10 to 130175
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   side                18448 non-null  int64  
 1   gamelength          18448 non-null  int64  
 2   result              18448 non-null  int64  
 3   firstdragon         18448 non-null  float64
 4   dragons             18448 non-null  float64
 5   elementaldrakes     18448 non-null  float64
 6   infernals           18448 non-null  float64
 7   mountains           18448 non-null  float64
 8   clouds              18448 non-null  float64
 9   oceans              18448 non-null  float64
 10  chemtechs           18448 non-null  float64
 11  hextechs            18448 non-null  float64
 12  elders              18448 non-null  float64
 13  firstherald         18448 non-null  float64
 14  heralds             18448 non-null  float64
 15  firstbaron          18448 non-null  float64
 16  barons 

In [84]:
# Look at the general values of the data. Note that popularity can be '0' - and there are many rows with that value
df_objectives.describe()

,side,gamelength,result,firstdragon,dragons,elementaldrakes,infernals,mountains,clouds,oceans,...,firstherald,heralds,firstbaron,barons,firsttower,towers,firstmidtower,firsttothreetowers,turretplates,inhibitors
count,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,...,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000,18448.000000
mean,0.500000,1882.021032,0.500000,0.499892,2.173135,2.131288,0.357003,0.360364,0.354618,0.355377,...,0.499892,0.992628,0.483955,0.744363,0.500000,6.058218,0.500000,0.500000,4.421672,0.943408
std,0.500014,337.730404,0.500014,0.500014,1.400907,1.355497,0.644343,0.650102,0.649259,0.648676,...,0.500014,0.781799,0.499756,0.771589,0.500014,3.676779,0.500014,0.500014,2.570972,1.110546
min,0.000000,976.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1647.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,3.000000,0.000000
50%,0.500000,1839.000000,0.500000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.500000,7.000000,0.500000,0.500000,4.000000,1.000000
75%,1.000000,2080.000000,1.000000,1.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,1.000000,9.000000,1.000000,1.000000,6.000000,2.000000
max,1.000000,3604.000000,1.000000,1.000000,6.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,1.000000,2.000000,1.000000,4.000000,1.000000,11.000000,1.000000,1.000000,15.000000,8.000000


In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a DataFrame called 'df' with the team stats and 'result' column indicating the winner (1=win, 0=loss)
# Split the data into training and testing sets
X = df_objectives.drop('result', axis=1)  # Features (team stats)
y = df_objectives['result']  # Target variable (winner)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9710027100271003


c:\Users\Ejer\Documents\Unity\MED7\ML-For-Beginners\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
# Assuming you have a DataFrame called 'df' with a column 'side' indicating the side of the game (e.g., 'blue' or 'red')
# and a column 'result' indicating the result of the game (1=win, 0=loss)

# Calculate the total number of games for each side
total_games = df_objectives.groupby('side').size()

# Calculate the number of wins for each side
wins = df_objectives[df_objectives['result'] == 1].groupby('side').size()

# Calculate the win percentage for each side
win_percentage = (wins / total_games) * 100

# Print the win percentage for each side
print(win_percentage)



side
Blue    53.465255
Red     46.538391
dtype: float64


In [25]:
#import wandb